# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015)

In [1]:
SQL_SOURCE = 'https://geeroovaa.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query}).encode('utf-8')
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-ab9982693636>, line 23)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT * 
FROM citibike
ORDER BY start_station_id
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT * 
FROM citibike
ORDER BY tripduration
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT * 
FROM citibike
WHERE tripduration < 10800
ORDER BY tripduration
FETCH FIRST 10 ROWS ONLY
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT *
FROM citibike
WHERE tripduration < 10800
ORDER BY tripduration DESC
FETCH FIRST 10 ROWS ONLY
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT DISTINCT start_station_id
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT COUNT (*)
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT MIN (tripduration)
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT MAX (tripduration)
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT AVG (tripduration)
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [2]:
test='''
SELECT * 
FROM citibike
WHERE starttime>='2015-02-02 00:00' AND starttime<'2015-02-03 00:00'
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

NameError: name 'pd' is not defined

In [ ]:
test='''
SELECT * 
FROM citibike
WHERE extract(DOW FROM starttime) IN (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT * FROM citibike
WHERE extract(DOW FROM starttime) NOT IN (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id
FROM citibike
GROUP BY
 start_station_latitude,
 start_station_longitude
 '''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT
 start_station_id,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 COUNT(tripduration) as numtrips
FROM citibike
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
  '''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT
 start_station_id,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 COUNT(tripduration) as numtrips
FROM citibike
WHERE
ST_DWithin(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
 '''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT
 start_station_id,
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude)
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 AVG(tripduration) as avg_duration
FROM citibike
WHERE
ST_DWithin(
 CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
AND
extract(DOW FROM starttime) IN (0,6)

GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude
ORDER BY avg_duration DESC
LIMIT 1
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
test='''
SELECT
 start_station_id,
 end_station_id,
 ST_Makeline(
 CDB_TransformToWebmercator(CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )),
 CDB_TransformToWebmercator(CDB_LatLng(
 end_station_latitude,
 end_station_longitude
 ))
 ) as the_geom_webmercator,
 MIN(cartodb_id) as cartodb_id,
 AVG(tripduration) as avg_duration
FROM citibike
WHERE
tripduration<7200
and
ST_DWithin(
CDB_LatLng(
 start_station_latitude,
 start_station_longitude
 )::geography,
 CDB_LatLng(40.7577,-73.9857)::geography,
 500)
GROUP BY
 start_station_id,
 start_station_latitude,
 start_station_longitude,
 end_station_id,
 end_station_latitude,
 end_station_longitude
 '''
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

## ASH Score - 100, EXtra credit 0 